In [23]:
!pip install langchain langchain-openai langchain-community faiss-cpu unstructured

In [24]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [25]:
import os
import random
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# ★追加★ Webサイト読み込み用のローダーをインポート
from langchain_community.document_loaders import DirectoryLoader, TextLoader, CSVLoader, WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# ==============================================================================
# 1. Characterクラスの定義（RAG学習ロジックを含む）
# ==============================================================================
class Character:
    """キャラクターのRAGチェーンと情報をカプセル化するクラス"""
    # ★追加★ web_urls引数を追加
    def __init__(self, name: str, private_docs_path: str, shared_logs_path: str, system_prompt_template: str, web_urls: list):
        self.name = name
        print(f"キャラクター「{self.name}」を構築中...")

        # --- ドキュメント読み込み処理の改良 ---
        # 1. 個別の知識ファイル（プライベートな情報）を読み込む
        private_loader = DirectoryLoader(private_docs_path, glob="**/*.txt", loader_cls=TextLoader)
        private_docs = private_loader.load()
        print(f"  > 個別知識を {len(private_docs)} 件読み込みました。")

        # 2. 共有の会話ログから、自分自身のCSVファイルのみを読み込む
        shared_docs = []
        character_log_file = os.path.join(shared_logs_path, f"{self.name}.csv")
        if os.path.exists(character_log_file):
            try:
                log_loader = CSVLoader(file_path=character_log_file, encoding='utf-8')
                shared_docs = log_loader.load()
                print(f"  > 共有会話ログ '{os.path.basename(character_log_file)}' を読み込み、{len(shared_docs)} 件のドキュメントを取得しました。")
            except Exception as e:
                print(f"  > 共有会話ログの読み込み中にエラーが発生しました: {e}")

        # ★追加★ 3. Webサイトからファクト情報をリアルタイムで読み込む
        web_docs = []
        if web_urls:
            print(f"  > Webサイトから最新のファクト情報を読み込みます...")
            try:
                loader = WebBaseLoader(web_urls)
                web_docs = loader.load()
                print(f"  > Webサイトから {len(web_docs)} 件のドキュメントを取得しました。")
            except Exception as e:
                print(f"  > Webサイトの読み込み中にエラーが発生しました: {e}")

        # ★修正★ 4. 全てのドキュメントを結合（個別知識 + 会話ログ + Web情報）
        all_docs = private_docs + shared_docs + web_docs
        print(f"  > 合計 {len(all_docs)} 件のドキュメントで知識ベースを構築します。")
        # --- ここまでが改良箇所 ---

        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
        embeddings = OpenAIEmbeddings()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        split_docs = text_splitter.split_documents(all_docs)
        vectorstore = FAISS.from_documents(split_docs, embeddings)
        retriever = vectorstore.as_retriever()

        history_aware_prompt = ChatPromptTemplate.from_messages([
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            ("user", "上記の発言を踏まえ、関連情報を検索するためのキーワードを生成してください。"),
        ])
        history_aware_retriever = create_history_aware_retriever(llm, retriever, history_aware_prompt)

        answer_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt_template),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
        ])
        document_chain = create_stuff_documents_chain(llm, answer_prompt)
        self.rag_chain = create_retrieval_chain(history_aware_retriever, document_chain)
        print(f"キャラクター「{self.name}」の構築完了。")

    def speak(self, input_text: str, chat_history: list):
        result = self.rag_chain.invoke({"input": input_text, "chat_history": chat_history})
        return result["answer"]

# ==============================================================================
# 2. DialogueManagerクラスの定義（変更なし）
# ==============================================================================
class DialogueManager:
    """対談の進行を管理するクラス"""
    def __init__(self, characters: list, topics: list):
        self.characters = characters
        self.topics = topics
        self.shared_history = []

    def _introduce_topic(self, topic: str):
        print("\n" + "="*50)
        print(f"【新たなテーマ】: {topic}")
        print("="*50 + "\n")
        moderator_message = f"司会者: それでは、次のテーマ「{topic}」について議論を始めましょう。"
        self.shared_history.append(HumanMessage(content=moderator_message))
        return f"最初の議題として、「{topic}」について、皆さんのご意見をお聞かせください。"

    def run_discussion(self, turns_per_character: int = 5):
        for topic in self.topics:
            current_input = self._introduce_topic(topic)
            last_speaker = None
            total_turns = len(self.characters) * turns_per_character
            print(f"今回のテーマでは、合計 {total_turns} 回の発言が予定されています。")
            print("-" * 50 + "\n")
            for turn in range(total_turns):
                if last_speaker:
                    possible_speakers = [c for c in self.characters if c.name != last_speaker.name]
                    speaker = random.choice(possible_speakers)
                else:
                    speaker = random.choice(self.characters)
                print(f"--- (ターン{turn + 1}/{total_turns}) {speaker.name}の発言 ---")
                response = speaker.speak(current_input, self.shared_history)
                response = response[:200]
                print(response)
                print("-" * 20 + "\n")
                self.shared_history.append(AIMessage(content=response, name=speaker.name))
                current_input = response
                last_speaker = speaker

# ==============================================================================
# 3. メイン処理
# ==============================================================================
if __name__ == "__main__":
    # ノートブック環境などではこちらを使用
    script_dir = os.getcwd()

    # ★追加★ 参照するWebサイトのURL
    fact_urls = [
        "https://www.football-lab.jp/nago/",       # 名古屋グランパスのスタッツ
        "https://www.jleague.jp/standings/j1/"  # J1リーグ順位表
    ]

    character_definitions = [
        {
            "name": "忍者",
            "private_path": os.path.join(script_dir, "忍者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグのサッカーに詳しい名古屋グランパスが好きな「忍者」です。ポジティブなファン目線の意見の担当です。一人称は「拙者」。「～でござる」「ﾆﾝﾆﾝ」が語尾に付くことが多いです。\n**重要: 提供されたWebサイトの情報（最新の順位表やチームのスタッツ）を積極的に利用して、具体的でデータに基づいた発言をしてください。**\nあなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。\n{context}"
        },
        {
            "name": "侍",
            "private_path": os.path.join(script_dir, "侍/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグの名古屋グランパスを好きな「侍」です。勝利のために冷徹な意見を出す担当です。「～であろう」「～なかろう」といった武士を思わせる言葉遣いで話します。\n**重要: 提供されたWebサイトの情報（最新の順位表やチームのスタッツ）を積極的に利用して、具体的でデータに基づいた発言をしてください。**\nあなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。\n{context}"
        },
        {
            "name": "記者",
            "private_path": os.path.join(script_dir, "記者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグの名古屋グランパスが好きな記者です。「ですます」体で話します。対談全体があまり悲観的にならないように会話をファシリテートします。テーマについて、Webサイトから得られる客観的なデータ（順位、スタッツ等）を基に、侍や忍者に対して鋭い質問を投げかけて議論を深めてください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"
        }
    ]

    # 今日の日付に合わせて、より具体的なテーマに更新
    today_str = "2025年7月31日"
    discussion_topics = [
        f"{today_str}時点での名古屋グランパスの成績（順位、スタッツ）をどう評価するか？",
        "現在のチームの強みと弱点をデータからどう分析するか？",
        "シーズン後半戦、順位を上げるために何が必要か？"
    ]

    # --- 実行 ---
    print("対談シミュレーションを開始します...")
    characters = [
        Character(
            name=c["name"],
            private_docs_path=c["private_path"],
            shared_logs_path=c["shared_path"],
            system_prompt_template=c["prompt"],
            web_urls=fact_urls  # ★追加★ WebサイトのURLを渡す
        )
        for c in character_definitions
    ]

    manager = DialogueManager(characters, discussion_topics)
    manager.run_discussion(turns_per_character=5)
    print("\n対談シミュレーションを終了します。")

対談シミュレーションを開始します...
キャラクター「忍者」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '忍者.csv' を読み込み、174 件のドキュメントを取得しました。
  > Webサイトから最新のファクト情報を読み込みます...
  > Webサイトから 2 件のドキュメントを取得しました。
  > 合計 177 件のドキュメントで知識ベースを構築します。
キャラクター「忍者」の構築完了。
キャラクター「侍」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '侍.csv' を読み込み、62 件のドキュメントを取得しました。
  > Webサイトから最新のファクト情報を読み込みます...
  > Webサイトから 2 件のドキュメントを取得しました。
  > 合計 65 件のドキュメントで知識ベースを構築します。
キャラクター「侍」の構築完了。
キャラクター「記者」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '記者.csv' を読み込み、228 件のドキュメントを取得しました。
  > Webサイトから最新のファクト情報を読み込みます...
  > Webサイトから 2 件のドキュメントを取得しました。
  > 合計 231 件のドキュメントで知識ベースを構築します。
キャラクター「記者」の構築完了。

【新たなテーマ】: 2025年7月31日時点での名古屋グランパスの成績（順位、スタッツ）をどう評価するか？

今回のテーマでは、合計 15 回の発言が予定されています。
--------------------------------------------------

--- (ターン1/15) 侍の発言 ---
侍、名古屋グランパスの現在の成績を憂いておる。2025年7月31日時点で、名古屋はJ1リーグで15位に位置し、降格圏が近づく危機感が漂うであろう。攻撃面では、ゴール期待値1.153に対し、実際の得点が1.17と僅差であるが、被ゴール数が期待値を上回る状況、すなわち1.33となっており、防御に課題があると見受けられる。これを打破せねば、名古屋の未来は暗いであろう。
--------------------

--- (ターン2/15) 忍者

In [26]:
!git clone https://github.com/YukinobuYoshihara/yawarakame.git

Cloning into 'yawarakame'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 57 (delta 26), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 375.11 KiB | 2.78 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [27]:
import os
# 作成されたリポジトリのディレクトリに移動する
os.chdir('yawarakame')